# -- Instruções de Uso --

Este notebook executa a extração, normalização e exportação dos dados DFP (Demonstrações Financeiras Padronizadas) disponibilizados pela CVM.

##Para utilizar:

1. Execute cada célula na ordem em que aparece.

2. Para executar uma célula, utilize Shift + Enter ou clique no ícone de play à esquerda.

3. A célula abaixo inicia o processo completo de extração (download dos arquivos, leitura, tratamento e geração dos CSVs).

4. Ao final, será disponibilizado um arquivo .zip contendo todos os demonstrativos consolidados em formato .csv.

5. Siga a sequência de execução até o final para gerar e baixar os dados corretamente.

In [3]:
import os

RAW_DIR  = "data_raw/cvm/dfp"
OUT_DIR  = "data_processed/cvm/dfp"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

import zipfile
import requests
import pandas as pd

BASE_DFP = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS"


# Função baixar

def baixar_zip_ano(year: int) -> str:
    url = f"{BASE_DFP}/dfp_cia_aberta_{year}.zip"
    local = os.path.join(RAW_DIR, f"dfp_{year}.zip")

    if not os.path.exists(local):
        print(f"[INFO] Baixando {url} ...")
        r = requests.get(url, timeout=150)
        r.raise_for_status()
        with open(local, "wb") as f:
            f.write(r.content)
        print(f"[OK] Salvo: {local}")
    else:
        print(f"[INFO] ZIP já existe: {local}")

    return local

#######

def ler_zip_csvs(local_zip: str) -> dict[str, pd.DataFrame]:
    dfs = {}
    with zipfile.ZipFile(local_zip) as z:
        for name in z.namelist():
            if not name.lower().endswith(".csv"):
                continue
            df = pd.read_csv(
                z.open(name),
                sep=";",
                encoding="latin1",
                dtype=str
            )
            df.columns = [c.strip().upper() for c in df.columns]
            dfs[name] = df
    return dfs


## Normalização

def aplicar_escala(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["VL_CONTA"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")

    multiplicador = df["ESCALA_MOEDA"].map({"MIL": 1000, "UNIDADE": 1})
    multiplicador = multiplicador.fillna(1)

    df["VL_CONTA_REAL"] = df["VL_CONTA"] * multiplicador
    return df


def limpar_basico(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "DT_REFER" in df.columns:
        df["DT_REFER"] = pd.to_datetime(df["DT_REFER"], errors="coerce")

    if "MOEDA" in df.columns:
        df = df[df["MOEDA"].str.upper() == "REAL"]

    colunas = [
        "CNPJ_CIA", "CD_CVM", "DENOM_CIA", "DT_REFER", "GRUPO_DFP",
        "ORDEM_EXERC", "CD_CONTA", "DS_CONTA",
        "ESCALA_MOEDA", "VL_CONTA", "VL_CONTA_REAL"
    ]

    return df[[c for c in colunas if c in df.columns]].copy()


def parse_dfp_ano(year: int) -> dict[str, pd.DataFrame]:
    local = baixar_zip_ano(year)
    dfs_raw = ler_zip_csvs(local)

    out = {
        "DRE": [],
        "DFC_MI": [],
        "DFC_MD": [],
        "BPA": [],
        "BPP": []
    }

    for name, df in dfs_raw.items():
        name_low = name.lower()

        if   "dre_con"     in name_low: grupo = "DRE"
        elif "dfc_mi_con"  in name_low: grupo = "DFC_MI"
        elif "dfc_md_con"  in name_low: grupo = "DFC_MD"
        elif "bpa_con"     in name_low: grupo = "BPA"
        elif "bpp_con"     in name_low: grupo = "BPP"
        else:
            continue

        df["GRUPO_DFP"] = grupo
        df = aplicar_escala(df)
        df = limpar_basico(df)

        out[grupo].append(df)

    for g in out:
        if out[g]:
            out[g] = pd.concat(out[g], ignore_index=True)
        else:
            out[g] = pd.DataFrame()

    return out


## Execução

def build_dfp(start_year=2010, end_year=2024):
    buckets = {"DRE": [], "DFC_MI": [], "DFC_MD": [], "BPA": [], "BPP": []}


    for year in range(start_year, end_year + 1):
        print(f"[ANO] {year}")
        ano_dfs = parse_dfp_ano(year)

        for g, df in ano_dfs.items():
            if not df.empty:
                buckets[g].append(df)


    for g, lst in buckets.items():
        if not lst:
            print(f"[WARN] Sem dados para {g}")
            continue

        full = pd.concat(lst, ignore_index=True)

        outpath = os.path.join(OUT_DIR, f"{g.lower()}_{start_year}_{end_year}.csv")

        full.to_csv(outpath, index=False, encoding="utf-8")

        print(f"[OK] {g} salvo em {outpath}")

    print("Todas as demonstrações foram exportadas para CSV.")


build_dfp(2010, 2024)


[ANO] 2010
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2010.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2010.zip
[ANO] 2011
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2011.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2011.zip
[ANO] 2012
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2012.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2012.zip
[ANO] 2013
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2013.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2013.zip
[ANO] 2014
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2014.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2014.zip
[ANO] 2015
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2015.zip ...
[OK] Salvo: data_raw/cvm/dfp/dfp_2015.zip
[ANO] 2016
[INFO] Baixando https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS

# -- BLOCO ABAIXO É PARA VALIDAÇÃO DOS DADOS BAIXADOS --

Em síntese, conferência dos dados, coerência analisadaa via tabelas síntese, e comparação contábil breve

In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

BASE = OUT_DIR  # mesma pasta dos CSVs gerados

FILES = {
    "DRE":    "dre_2010_2024.csv",
    "BPA":    "bpa_2010_2024.csv",
    "BPP":    "bpp_2010_2024.csv",
    "DFC_MI": "dfc_mi_2010_2024.csv",
    "DFC_MD": "dfc_md_2010_2024.csv",
}

dfs = {}

print("Carregando arquivos CSV...")

for nome, fname in FILES.items():
    path = os.path.join(BASE, fname)
    if os.path.exists(path):
        df = pd.read_csv(path)
        dfs[nome] = df
        print(f"  ✔️ {nome}: {len(df):,} linhas")
    else:
        print(f"  ⚠️ {nome} não encontrado")


# ============================================================
# HEAD CURTO POR DEMONSTRAÇÃO
# ============================================================

print("\n### Visualização inicial (head curto) ###")

for nome, df in dfs.items():
    print(f"\n===== {nome} =====")
    display(df.head(3))


# ============================================================
# RESUMO ESTRUTURAL E DESCRITIVO
# ============================================================

print("\n### Resumo estatístico e estrutural ###")

for nome, df in dfs.items():

    print(f"\n===== {nome} =====")

    # Quantidade de linhas
    print(f"Linhas totais: {len(df):,}")

    # Datas
    if "DT_REFER" in df.columns:
        try:
            df["DT_REFER"] = pd.to_datetime(df["DT_REFER"], errors='coerce')
            print("Período:", df["DT_REFER"].min().date(), "→", df["DT_REFER"].max().date())
        except:
            print("Período: não foi possível interpretar datas")

    # Empresas únicas
    if "CD_CVM" in df.columns:
        print("Empresas únicas:", df["CD_CVM"].nunique())

    # Estatística VL_CONTA_REAL
    if "VL_CONTA_REAL" in df.columns:
        print("\nEstatística básica (VL_CONTA_REAL):")
        display(df["VL_CONTA_REAL"].describe(percentiles=[.01, .25, .5, .75, .99]))


# ============================================================
# TESTE CONTÁBIL: ATIVO TOTAL vs PASSIVO TOTAL
# ============================================================

print("\n### Teste contábil (Ativo Total ≈ Passivo Total) ###")

if "BPA" in dfs and "BPP" in dfs:

    bpa = dfs["BPA"]
    bpp = dfs["BPP"]

    atv = bpa[bpa["DS_CONTA"].str.contains("Ativo Total", case=False, na=False)].copy()
    pas = bpp[bpp["DS_CONTA"].str.contains("Passivo Total", case=False, na=False)].copy()

    # Converte data
    atv["DT_REFER"] = pd.to_datetime(atv["DT_REFER"], errors="coerce")
    pas["DT_REFER"] = pd.to_datetime(pas["DT_REFER"], errors="coerce")

    merged = pd.merge(
        atv,
        pas,
        on=["CD_CVM", "DT_REFER"],
        suffixes=("_ATV", "_PAS")
    )

    merged["dif"] = merged["VL_CONTA_REAL_ATV"] - merged["VL_CONTA_REAL_PAS"]

    print("Diferença média:", merged["dif"].mean())
    print("Diferença absoluta média:", merged["dif"].abs().mean())
    print("Percentil 99 (abs):", merged["dif"].abs().quantile(0.99))

else:
    print("BPA ou BPP indisponíveis.")



Carregando arquivos CSV...
  ✔️ DRE: 418,738 linhas
  ✔️ BPA: 761,534 linhas
  ✔️ BPP: 1,295,608 linhas
  ✔️ DFC_MI: 564,947 linhas
  ✔️ DFC_MD: 8,066 linhas

### Visualização inicial (head curto) ###

===== DRE =====


,CNPJ_CIA,CD_CVM,DENOM_CIA,DT_REFER,GRUPO_DFP,ORDEM_EXERC,CD_CONTA,DS_CONTA,ESCALA_MOEDA,VL_CONTA,VL_CONTA_REAL
0,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,DRE,PENÚLTIMO,3.01,Receitas da Intermediação Financeira,MIL,67608506.0,6.760851e+10
1,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,DRE,ÚLTIMO,3.01,Receitas da Intermediação Financeira,MIL,85143206.0,8.514321e+10
2,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,DRE,PENÚLTIMO,3.01.01,Receita de Juros,MIL,67608506.0,6.760851e+10



===== BPA =====


,CNPJ_CIA,CD_CVM,DENOM_CIA,DT_REFER,GRUPO_DFP,ORDEM_EXERC,CD_CONTA,DS_CONTA,ESCALA_MOEDA,VL_CONTA,VL_CONTA_REAL
0,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,BPA,PENÚLTIMO,1,Ativo Total,MIL,702571987.0,7.025720e+11
1,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,BPA,ÚLTIMO,1,Ativo Total,MIL,802819794.0,8.028198e+11
2,00.000.000/0001-91,1023,BCO BRASIL S.A.,2010-12-31,BPA,PENÚLTIMO,1.01,Caixa e Equivalentes de Caixa,MIL,31037881.0,3.103788e+10



===== BPP =====


...


===== DFC_MI =====
Linhas totais: 564,947
Período: 2010-12-31 → 2024-12-31
Empresas únicas: 705

Estatística básica (VL_CONTA_REAL):


,VL_CONTA_REAL
count,5.649470e+05
mean,1.164777e+08
std,5.103761e+09
min,-3.978653e+11
1%,-3.342297e+09
25%,-8.781000e+06
50%,0.000000e+00
75%,1.219800e+07
99%,5.053577e+09
max,4.409720e+11



===== DFC_MD =====
Linhas totais: 8,066
Período: 2010-12-31 → 2024-12-31
Empresas únicas: 49

Estatística básica (VL_CONTA_REAL):


,VL_CONTA_REAL
count,8.066000e+03
mean,2.309801e+08
std,7.663119e+09
min,-2.192944e+11
1%,-6.112531e+09
25%,-7.725000e+06
50%,0.000000e+00
75%,9.505000e+06
99%,9.705450e+09
max,1.908210e+11



### Teste contábil (Ativo Total ≈ Passivo Total) ###
Diferença média: -0.05656872678149274
Diferença absoluta média: 2005332481.7319317
Percentil 99 (abs): 36642969000.0


---

## Download dos arquivos CSV (DFP 2010–2024)

O bloco abaixo irá:

1. Compactar todos os arquivos CSV gerados (`DRE`, `DFC_MI`, `DFC_MD`, `BPA`, `BPP`)
2. Criar um arquivo único: **dfp_2010_2024_csvs.zip**
3. Exibir o botão de download automaticamente no Colab

### Nota sobre permissões no navegador
Dependendo do navegador (Chrome, Edge, Firefox), pode aparecer um aviso pedindo permissão para baixar arquivos.

Basta clicar em **"Permitir"**, e o download será iniciado.


---

Aperte **Shift+Enter** na célula abaixo para gerar e baixar o `.zip`.

---


In [8]:
# Compacta todos os CSVs em um único arquivo ZIP
import shutil

zip_path = "dfp_2010_2024_csvs.zip"
shutil.make_archive("dfp_2010_2024_csvs", 'zip', OUT_DIR)

# Gera botão funcional
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>